# Import

In [ ]:
bucket <- "gs://fc-secure-8c0f9180-d3ff-4f0d-8299-78e01606927c"
filenames <- c("t2d_f9_sgaynor.4.24_harmonized_postprocessed.csv",
               "common_covariates_PIC-SURE_12-9-2021_AKM.csv",
               "ancestry.supervised.fhs.mesa.whi.xlsx",
               "WHI_omics_technical_phenotypes_2021-05-25_good.csv",
               "omics_sample_metadata_WHI_RNAseq.xlsx",
               "omics_sample_metadata_WHI_metabolomics.csv",
               "TOPMed_MESA_RNAseq_sample_attributes_freeze.xlsx",
               "MESA_TOPMed_Metab_Proteo_BySidnoExam.csv",
               "meth_metab_timepoint_visit_info_17FEB2022.csv",
               "WHI_omics_technical_phenotypes_2021-05-25_good_added_bmi.csv")
filepaths <- paste0(bucket,'/',filenames)
sapply(filepaths, function(filepath) system(paste("gsutil cp",filepath,"./ 2>&1")) ) # If 1, file failed to copy

# Read

In [ ]:
library(readxl)
library(data.table)
# "bdl" stands for "BIG DF LIST!", a list of all those clinical variable dfs together.
bdl <- lapply(filenames, function(filename) {
    if(grepl(".csv", filename)) return(read.csv(filename))
    if(grepl(".xlsx",filename)) return(read_excel(filename))
})
names(bdl) <- filenames

In [ ]:
 # Inspect
lapply(bdl,dim)
lapply(bdl,head)

# Keep only needed cols

In [ ]:
bdl[[1]] <- bdl[[1]][bdl[[1]]$study %in% c("MESA","WHI"),
                      c("sample.id", # NWD
                        "subject_id", # DBG
                        "sex",
                        "ancestry", # AF AS EU HS SAS. "population" col is an identical()==TRUE copy of this.
                                      # NOTE: compare this with "population" in kent's file (bdl[[3]]) to make sure they match
                        "topmedid", # NWD
                        "t2d", # 0, 1, or 2
                        "t2d_age",
                        "t2d_bmi")]
bdl[[2]] <- bdl[[2]][bdl[[2]]$`X._VCF.Sample.Id.`!="",
                      c("X.DCC.Harmonized.data.set.03...Baseline.common.covariates.Indicates.whether.subject.ever.regularly.smoked.cigarettes..",
                        # ^ Current or former cigarette smoker", or "Never a cigarette smoker", or blank "" (factor)
                        "X._VCF.Sample.Id.")] # NWD
bdl[[2]] <- bdl[[2]][complete.cases(bdl[[2]]),]
bdl[[3]] <- bdl[[3]][bdl[[3]]$study %in% c("WHI","MESA"), # "population" & "group" are empty for WHI/MESA entries.
                      c("index", # "index" & "subject" are identical, just picked one
                        "europe",
                        "african",
                        "american",
                        "east_asian",
                        "study")]
bdl[[4]] <- bdl[[4]][,c("subject_id", # "7#####" WHI-specific ID
                        "olink_id", # (just for proteomics)
                        "rnaseq_id",      # TOR
                        "methylation_id", # TOE
                        "TOM_ID",         # TOM
                        "methylation_batch", # 14 batches "B0001"-"B0014" and also a blank "" batch.
                        "methylation_chip", # just a constant: 2.03e+11
                        "methylation_row", # 8 batches "R01"-"RO8" and also a blank "" batch.
                        "rnaseq_batch",
                        "draw_age")]
bdl[[5]] <- bdl[[5]][,c("SUBJECT_ID", # WHI-specific "7######" IDs (RNASeq)
                        "SAMPLE_ID",  # TOR
                        "Subject_fasting_hours")]
bdl[[6]] <- bdl[[6]][bdl[[6]]$Collection_visit!="Baseline",
                     c("SUBJECT_ID", # WHI-specific "7######" IDs (metabolomics)
                        "SAMPLE_ID",  # TOM
                        "Subject_fasting_hours")]
bdl[[7]] <- bdl[[7]][bdl[[7]]$exam==1, # MESA RNASeq
                      c("tor_id", # TOR
                        "nwd_id", # NWD
                        "exclusion_reason")]
bdl[[8]] <- bdl[[8]][bdl[[8]]$Exam==1, # MESA metab & proteo
                      c("sidno", # "1###" IDs
                        "TOM_ID", # TOM
                        "TOP_ID", # TOP
                        "collaboratorsampleid", # NWD
                        "ageatdraw",
                        "bmi1c")]
# Just keep all of bdl[[9]] & bdl[[10]]
lapply(bdl,dim) # Confim # of cols was cut down
lapply(bdl,head)

In [ ]:
# Just exploring WHI duplicates
# 0 (so MESA & WHI TORs don't conflict): sum(bdl[[5]]$SAMPLE_ID %in% bdl[[7]]$tor_id)
sum(duplicated(bdl[[4]]$rnaseq_id))
sum(duplicated(bdl[[4]]$subject_id))
sum(duplicated(bdl[[5]]$SUBJECT_ID))
sum(duplicated(bdl[[5]]$SAMPLE_ID))
# 64 bdl[[7]][duplicated(bdl[[7]]$nwd_id),]
# 0 sum(duplicated(bdl[[7]]$nwd_id) & duplicated(bdl[[7]]$tor_id))

dim(bdl[[4]])
dim(bdl[[5]])
head(bdl[[5]]$SAMPLE_ID)

# look @ bdl 4 & 5 and if there are two rows which are identical (excluding NAs) except for the TOR, then pick the one with the fewer NAs.

# Now exploring MESA duplicates?
tmp <- bdl[[8]]$TOP_ID[duplicated(bdl[[8]]$TOP_ID)]
tmp
tmp <- bdl[[8]]$TOM_ID[duplicated(bdl[[8]]$TOM_ID)]
tmp
# Phew, they are just blank strings (NAs)

# MESA-only File

## Merge!

In [ ]:
# bdl[c(4,5,6,9,10)] are WHI-only, needn't bother
# All of these merges concern NWD IDs.
mesa_df <- merge(x=bdl[[1]], by.x="sample.id",
          y=merge(x=bdl[[2]], by.x="X._VCF.Sample.Id.",
           y=merge(x=bdl[[3]], by.x="index",
            y=merge(x=bdl[[7]], by.x="nwd_id",
                     y=bdl[[8]], by.y="collaboratorsampleid", all=T),
                                by.y="nwd_id", all=T), 
                               by.y="index", all=T),
                              by.y="X._VCF.Sample.Id.", all=T)
mesa_df <- setDT(mesa_df)
dim(mesa_df); head(mesa_df)

## Remove uninformative rows, Merge/Remove columns

### CHECKPOINT: check random rows

In [ ]:
rind <- trunc( runif(1,1,nrow(mesa_df)) ); rind
print(mesa_df[rind,])
dim(mesa_df)

### Remove rows w/o any multi-omic data

In [ ]:
dim(mesa_df)
mesa_df <- mesa_df[!is.na(tor_id) | !is.na(TOM_ID) | !is.na(TOP_ID)]
dim(mesa_df)

### Replace "study" column with "Cohort" column
"study" was from a merged file, so there might be some NAs. Want to guarantee it is a solid column of "MESA"s (to help in case this MESA file is ever merged with another clinical data file)

In [ ]:
mesa_df[, study := NULL]
mesa_df[, Cohort := "MESA"]

### "sample.id" vs. "topmedid"

In [ ]:
sum(is.na(mesa_df$sample.id)); sum(is.na(mesa_df$topmedid))
identical(mesa_df[!is.na(topmedid), sample.id],
          mesa_df[!is.na(topmedid), topmedid])
all(is.na( mesa_df[is.na(sample.id)]$topmedid ))
# They are identical except topmedid is less complete. Just keep sample.id then.
mesa_df[, topmedid := NULL]

### "subject_id" vs. "sidno"

In [ ]:
sum(is.na(mesa_df$subject_id)); sum(is.na(mesa_df$sidno))
mesa_df[!is.na(sidno), sidno,subject_id][sidno!=subject_id] # Noooo! They (exluding NAs) are identical except for ~one~ conflict!
# Asked Magda; will trust the sgaynor file ("subject_id")
# However, sidno does have a couple ids where subject_id is NA, so grab those.
mesa_df[is.na(subject_id) & !is.na(sidno), sidno]
mesa_df[is.na(subject_id), subject_id:=sidno]

mesa_df[, sidno := NULL]

### CHECKPOINT: check random rows

In [ ]:
rind <- trunc( runif(1,1,nrow(mesa_df)) ); rind
print(mesa_df[rind,])
dim(mesa_df)

## Prettier Colnames

In [ ]:
colnames(mesa_df)
colnames(mesa_df) <- c("NWD_Id", # Renaming
                       "Cohort_Specific_Id",
                       "Sex",
                       "Ancestry",
                       "T2D",
                       "T2D_Age",
                       "T2D_BMI",
                       "Smoking_Status",
                       "European",
                       "African",
                       "American",
                       "East_Asian",
                       "TOR_Id",
                       "Exclusion_Reason",
                       "TOM_Id",
                       "TOP_Id",
                       "Age_at_Draw",
                       "BMI",
                       "Cohort")
colnames(mesa_df)
mesa_df <- mesa_df[, c("Cohort_Specific_Id", # Rearranging
                       "NWD_Id",
                       "TOM_Id",
                       "TOP_Id",
                       "TOR_Id",
                       "Cohort",
                       "Sex",
                       "Ancestry",
                       "African",
                       "American",
                       "East_Asian",
                       "European",
                       "Smoking_Status",
                       "Age_at_Draw",
                       "T2D",
                       "T2D_Age",
                       "T2D_BMI",
                       "BMI",
                       "Exclusion_Reason")]
colnames(mesa_df)

## Final inspection (10 random rows)
Feel free to press ctrl+enter many times to cycle through many random samples, and make sure everything looks alright.

In [ ]:
dim(mesa_df)
mesa_df[runif(10,1,nrow(mesa_df))]

# WHI-only File

## Merge!

In [ ]:
# bdl[7:8] are MESA-only, needn't bother.
# What maps what IDs:
  # bdl[[1]]: NWD <-> WHI
  # bdl[2:3]: NWD
  # bld[[4]]:         WHI <-> TOE <-> TOM <-> TOR
  # bld[[5]]:         WHI <->                 TOR
  # bld[[6]]:         WHI <->         TOM
  # bld[[9]]:         WHI <-> TOE <-> TOM
  # bld[[10]]:        WHI
whi_df <- merge(x=bdl[[10]],by.x="subject_id", all=T,  # WHI
       y=merge(x=bdl[[9]], by.x="subject_id", all=T,  # WHI
      y=merge(x=bdl[[6]], by.x="SUBJECT_ID", all=T,  # WHI
     y=merge(x=bdl[[5]], by.x="SUBJECT_ID", all=T,  # WHI
    y=merge(x=bdl[[4]], by.x="subject_id", all=T,  # WHI
   y=merge(x=bdl[[3]], by.x="index",      all=T,  # NWD
  y=merge(x=bdl[[2]], by.x="X._VCF.Sample.Id.",  # NWD
         y=bdl[[1]],  by.y="sample.id", all=T),  # NWD
                        by.y="X._VCF.Sample.Id."),# NWD
                         by.y="subject_id"),       # WHI
                          by.y="subject_id"),       # WHI
                           by.y="SUBJECT_ID"),       # WHI
                            by.y="SUBJECT_ID"),       # WHI
                             by.y="subject_id")        # WHI
whi_df <- setDT(whi_df)
dim(whi_df); head(whi_df)

## Remove uninformative rows, Merge/Remove columns

### CHECKPOINT: Check a random row

In [ ]:
rind <- trunc( runif(1,1,nrow(whi_df)) ); rind
print(whi_df[rind,])

### Review what is in vaguely-named "ID" columns

In [ ]:
colnames(whi_df)
head( whi_df[!is.na(subject_id),  subject_id] )
head( whi_df[!is.na(SAMPLE_ID.x), SAMPLE_ID.x] )
head( whi_df[!is.na(SAMPLE_ID.y), SAMPLE_ID.y] )
head( whi_df[!is.na(index), index] )
head( whi_df[!is.na(topmedid), topmedid] )
head( whi_df[!is.na(olink_id), olink_id])

### Remove rows w/o any multi-omic data

In [ ]:
dim(whi_df)
whi_df <- whi_df[!(is.na(methylaiton_id) & is.na(methylation_id) &
                   is.na(metabolomics_id) & is.na(SAMPLE_ID.x) & is.na(TOM_ID) &
                   is.na(rnaseq_id) & is.na(SAMPLE_ID.y)
                  )]
dim(whi_df)

### "index" vs. "topmedid" (NWD IDs)

In [ ]:
identical(whi_df[!is.na(topmedid), index],
          whi_df[!is.na(topmedid), topmedid])
all(is.na( whi_df[is.na(index), topmedid] ))

whi_df[, topmedid := NULL]
# "topmedid" non-NAs are all identical to "index", just less complete.

### Remove Baseline TOE/TOM IDs 
Using "methylation_visit" and "metabolomics_visit" from Jeff's file

In [ ]:
table(whi_df$methylation_visit)
table(whi_df$metabolomics_visit)

# Set the TOE/TOM IDs associated w/ baseline to NA, omitting them.
sum(is.na(whi_df$methylation_id)); sum(is.na(whi_df$methylaiton_id));
whi_df[methylation_visit=="Baseline", `:=`(methylation_id=NA, methylaiton_id=NA)] # Not a mistake, there is a 2nd mispelled "methylaiton_id" col
sum(is.na(whi_df$methylation_id)); sum(is.na(whi_df$methylaiton_id)); # Confirm correct amount of additional NAs

sum(is.na(whi_df$SAMPLE_ID.x)); sum(is.na(whi_df$TOM_ID));
whi_df[metabolomics_visit=="Base",    `:=`(SAMPLE_ID.x=NA, TOM_ID=NA)]
sum(is.na(whi_df$SAMPLE_ID.x)); sum(is.na(whi_df$TOM_ID));

# "visit" cols no longer needed
whi_df[, methylation_visit := NULL]
whi_df[, metabolomics_visit := NULL]

###  "TOM_ID" vs. "SAMPLE_ID.x" vs. "metabolomics_id" (TOM IDs)

In [ ]:
identical(whi_df[!is.na(TOM_ID) & !is.na(SAMPLE_ID.x), TOM_ID],
          whi_df[!is.na(TOM_ID) & !is.na(SAMPLE_ID.x), SAMPLE_ID.x]) # :] can safely merge
identical(whi_df[!is.na(TOM_ID) & !is.na(metabolomics_id), TOM_ID],
          whi_df[!is.na(TOM_ID) & !is.na(metabolomics_id), metabolomics_id]) # :] and these too!

whi_df[is.na(TOM_ID), TOM_ID := SAMPLE_ID.x]
whi_df[is.na(TOM_ID), TOM_ID := metabolomics_id]

whi_df[, SAMPLE_ID.x := NULL] # merged into "TOM_ID", now redundant
whi_df[, metabolomics_id := NULL]

### "methylation_id" vs. "methylaiton_id" [sic]

In [ ]:
identical(whi_df[!is.na(methylation_id) & !is.na(methylaiton_id), methylation_id],
          whi_df[!is.na(methylation_id) & !is.na(methylaiton_id), methylaiton_id]) # :] can safely merge

whi_df[is.na(methylation_id), methylation_id := methylaiton_id]
whi_df[, methylaiton_id := NULL]

### "rnaseq_id" (from WHI tech phenotype) vs. "SAMPLE_ID.y" (WHI RNASeq)

In [ ]:
identical(whi_df[!is.na(rnaseq_id) & !is.na(SAMPLE_ID.y), rnaseq_id],
          whi_df[!is.na(rnaseq_id) & !is.na(SAMPLE_ID.y), SAMPLE_ID.y]) # :(
head(whi_df[rnaseq_id!=SAMPLE_ID.y, c("subject_id","rnaseq_id","SAMPLE_ID.y")])
length(unique(whi_df$rnaseq_id))
length(unique(whi_df$SAMPLE_ID.y))
unique(whi_df$rnaseq_id)[which(!( unique(whi_df$rnaseq_id) %in% unique(whi_df$SAMPLE_ID.y) ))]  # just 1 TOR ID unique to "rnaseq_id", and it's "".
# So then it's OK to toss rnaseq_id, because SAMPLE_ID.y already contains all of rnaseq_id's unique possibilities and then some.
  # It was just those ""s in rnaseq_id which made me think both columns had important info which the other did not.
whi_df[, rnaseq_id := NULL]

In [ ]:
dim(whi_df)
whi_df <- unique(whi_df) # remove duplicate rows
dim(whi_df)

In [ ]:
# Simplified the two conflicting rows to one, but still duplicates... what's different about these rows w/ the same TOR?
whi_df[1:2]
identical(whi_df[1,],whi_df[2,])
print(whi_df[1:2,])
# Hm, only difference is one has a blank rnaseq_batch. So...
whi_df <- whi_df[!duplicated(whi_df[,!"rnaseq_batch"])]
dim(whi_df) # Wow that helped a lot! Down to 1807 rows.

#### Example of troubling duplications
Checking against the expression data

In [ ]:
system("gsutil cp gs://fc-secure-8c0f9180-d3ff-4f0d-8299-78e01606927c/data_TOPMed_Kooperberg_P4.RSEMv1.3.1_gene_expected_count.txt.gz ./ 2>&1")
install.packages("R.utils")
rna <- fread("data_TOPMed_Kooperberg_P4.RSEMv1.3.1_gene_expected_count.txt.gz")

tmp <- unique( whi_df[duplicated(SAMPLE_ID.y), SAMPLE_ID.y] ) # unique(duplicated TOR IDs)
sum(tmp %in% colnames(rna))
length(tmp)
# 10 TORs which have duplications are not in the RNASeq expression and could thus be removed.
  # This does not actually help choose between the duplicated rows which remain though.
  # I was mostly hoping ~none~ of the duplicated TORs would be in the expression data so they could just all be removed.

In [ ]:
print(whi_df[SAMPLE_ID.y=="TOR482368"][c(1,4)])
# I see a couple rows could exclude for having more NAs
# But between the 1st and 4th row, it is still not clear which one to choose
# Pick based on whether TOM716939 or TOM222404 is in the metabolite expression data?
system("gsutil cp gs://fc-secure-8c0f9180-d3ff-4f0d-8299-78e01606927c/QCd_feb2022/QCd_merged/l2/WHI_met_QCd_l2.csv . 2>&1")
met <- fread("WHI_met_QCd_l2.csv")
"TOM716939" %in% colnames(met)
"TOM222404" %in% colnames(met)

identical(met[,"TOM716939"], met[,"TOM222404"])
# The expression data is not the same either, so cannot be merged.
# So then, is there nothing left to do but choose a row randomly?

In [ ]:
sum(duplicated(whi_df$subject_id))
sum(duplicated(whi_df$SAMPLE_ID.y))
sum(duplicated(whi_df[!duplicated(subject_id), SAMPLE_ID.y])) # oh no, even removing all duplicate WHI IDs there are still TOR IDs duplicated between different WHI IDs!! :(
whi_df[!duplicated(subject_id) & duplicated(SAMPLE_ID.y), c("subject_id", "SAMPLE_ID.y", "methylation_id", "TOM_ID")] # Which are they?
# Oh phew! They are just NAs. 
sum(duplicated(whi_df[!duplicated(subject_id), TOM_ID])) # same for TOM IDs! Hopefully these duplicates are just NAs as well?
whi_df[!duplicated(subject_id) & duplicated(TOM_ID), c("subject_id", "SAMPLE_ID.y", "methylation_id", "TOM_ID")] # Yes!

#### For duplicate WHI IDs, choose the instance with the least NAs
If two rows have the same number of NAs, then randomly choose one (with random seed == 1 for reproducibility). Random choice is done by sorting the rows randomly beforehand, then only choosing the first duplicate with the least NAs.

In [ ]:
dup_subj_ids <- unique( whi_df[duplicated(subject_id), subject_id] ) # unique(duplicated WHI IDs)
for(id in dup_subj_ids) {
    nas_per_instance <- apply(whi_df[subject_id==id],1, function(row) sum(is.na(row))) #print(nas_per_instance)
    rows_with_more_nas_than_min <- apply(whi_df,1, function(row) sum(is.na(row)) > min(nas_per_instance) )
    whi_df[subject_id==id & rows_with_more_nas_than_min, subject_id := "REMOVE"]
}
whi_df <- whi_df[subject_id!="REMOVE"]
sum(duplicated(whi_df[, subject_id]))
dim(whi_df)

### For the remaining duplicates, choose a random one
There are no other criteria which would make it clear which instance of the duplicate should be chosen. So we choose randomly (with a random seed == 1 for reproducibility).\
To do this, we randomly sort whi_df's rows, and then remove all duplicates but the first.

In [ ]:
set.seed(1)
whi_df <- whi_df[sample(1:nrow(whi_df))]
whi_df <- whi_df[!duplicated(subject_id)]
dim(whi_df)

In [ ]:
sum(duplicated(whi_df[,subject_id])) # Finally!
head(whi_df)

### Last thing: cohort/study column

In [ ]:
whi_df$study <- "WHI"

### CHECKPOINT: Check a random row

In [ ]:
rind <- trunc( runif(1,1,nrow(whi_df)) ); rind
print(whi_df[rind,])
colnames(whi_df)

## Prettier colnames

In [ ]:
colnames(whi_df) <- c("Subject_Id",
                      "BMI",
                      "BMI_Age",
                      "Subject_Fasting_Hours_WHI_Metabolomics",
                      "TOR_Id",
                      "Subject_Fasting_Hours_WHI_RNASeq",
                      "Olink_Id",
                      "TOE_Id",
                      "TOM_Id",
                      "Methylation_Batch",
                      "Methylation_Chip",
                      "Methylation_Row",
                      "RNASeq_Batch",
                      "Age_at_Draw",
                      "NWD_Id",
                      "European",
                      "African",
                      "American",
                      "East_Asian",
                      "Cohort",
                      "Smoking_Status",
                      "Sex",
                      "Ancestry",
                      "T2D",
                      "T2D_Age",
                      "T2D_BMI")
colnames(whi_df)

## Final inspection (10 random rows)
Feel free to press ctrl+enter many times to cycle through many random samples, and make sure everything looks alright.

In [ ]:
dim(whi_df)
whi_df[runif(10,1,nrow(whi_df))]

# Write

In [ ]:
dir.create(file.path("clinical_variables_and_ids/"))
#write.csv(mesa_df, file="clinical_variables_and_ids/clin_MESA_v2.csv", row.names=F)
write.csv(whi_df,  file="clinical_variables_and_ids/clin_WHI_v2.csv", row.names=F)
#write.csv(clin_df, file="clinical_variables_and_ids/clin_v1.csv", row.names=F)

In [ ]:
system(paste("gsutil cp -R clinical_variables_and_ids", bucket))

# (Ignore) Messy exploration
Ignore me. Just checking all the colnames of all the files to see which variables we want.

Notes: make one big map file for WHI and MESA separately. No FHS.\
only keep exam1\
make sure to keep the exclusion critera column\

## Import

In [ ]:
bucket <- "gs://fc-secure-8c0f9180-d3ff-4f0d-8299-78e01606927c"
filenames <- c("t2d_f9_sgaynor.4.24_harmonized_postprocessed.csv", # sex, anc, bmi
               "common_covariates_PIC-SURE_12-9-2021_AKM.csv", # smoking
               "ancestry.supervised.fhs.mesa.whi.xlsx", # anc
               "MESA_agebl_bmil_20210604.csv", # age @ draw
               "WHI_omics_technical_phenotypes_2021-05-25_good.csv",
               "ids_whi_olink.csv",
               "omics_sample_metadata_WHI_RNAseq.xlsx",
               "omics_sample_metadata_WHI_metabolomics.csv",
               "TOPMed_MESA_RNAseq_sample_attributes_freeze.xlsx",
               "MESA_TOPMed_Metab_Proteo_BySidnoExam.csv")
filepaths <- paste0(bucket,'/',filenames)
sapply(filepaths, function(filepath) system(paste("gsutil cp",filepath,"./ 2>&1")) )

## Inspect

In [ ]:
library(data.table)
sgaynor <- fread("t2d_f9_sgaynor.4.24_harmonized_postprocessed.csv")
colnames(sgaynor)
head(sgaynor)
sgaynor$individual_id
sgaynor <- sgaynor[,c("sample.id","subject_id","study","sex","ancestry","population","topmedid","t2d","t2d_age","t2d_bmi")]
table(sgaynor$ancestry)
table(sgaynor$population)
table(sgaynor$t2d)
table(sgaynor$study)
identical(sgaynor$ancestry, sgaynor$population)
sgaynor$topmedid[runif(20,1,nrow(sgaynor))]
sgaynor$subject_id

In [ ]:
smoking <- read.csv("common_covariates_PIC-SURE_12-9-2021_AKM.csv")
colnames(smoking)
dim(smoking)
head(smoking)
smoking <- smoking[,c("X.DCC.Harmonized.data.set.03...Baseline.common.covariates.Indicates.whether.subject.ever.regularly.smoked.cigarettes..",
                      "X._VCF.Sample.Id.")]
table(smoking$`X.DCC.Harmonized.data.set.03...Baseline.common.covariates.Indicates.whether.subject.ever.regularly.smoked.cigarettes..`)
# Note: VCF is NWD#####

In [ ]:
smoking[runif(1,1,nrow(smoking)), ]

In [ ]:
tmp <- read.csv("MESA_agebl_bmil_20210604.csv")
dim(tmp)
colnames(tmp)
head(tmp)

In [ ]:
library(readxl)
anc_supervised <- read_excel("ancestry.supervised.fhs.mesa.whi.xlsx")
dim(anc_supervised)
colnames(anc_supervised)
head(anc_supervised)
anc_supervised <- anc_supervised[,c("subject", "study", "europe", "african", "american", "east_asian")]

table(anc_supervised$study) # just grab MESA & WHI. You will notice MESA records have the NWD##### IDs
head(anc_supervised$subject[anc_supervised$study=="MESA"])
head(anc_supervised$subject[anc_supervised$study=="WHI"])

In [ ]:
tmp <- read.csv("WHI_omics_technical_phenotypes_2021-05-25_good.csv")
dim(tmp)
colnames(tmp)
head(tmp)
# want: subject_id   olink_id   rnaseq_id'methylation_id   TOM_ID   methylation_batch  draw_age
#         ^7###### are WHI-specific ids, helpful to map to NWDs
# oLINK ID IS  for proteomics
table(tmp$methylation_batch)
head(tmp$ETHNIC)

In [ ]:
tmp <- read_excel("omics_sample_metadata_WHI_RNAseq.xlsx")
dim(tmp)
colnames(tmp)
head(tmp)
# want: SUBJECT_ID, SAMPLE_ID, Subject_fasting_hours
    #      ^ WHI-specific id 7#### or 8######

In [ ]:
tmp <- read.csv("omics_sample_metadata_WHI_metabolomics.csv")
dim(tmp)
colnames(tmp)
head(tmp)
# want: SUBJECT_ID, Subject fasting hours, SAMPLE_ID
# same as above but metabolites
# maybe make sure that these fasting hours lines up with the above 

In [ ]:
tmp <- read_excel("TOPMed_MESA_RNAseq_sample_attributes_freeze.xlsx")
dim(tmp)
colnames(tmp)
#table(tmp$cohort)
#table(tmp$cohort2)
head(tmp[tmp$exam==1,])
#tmp$participant_id
head(tmp)
head(tmp$participant_id)
# want: tor_id, exam, nwd_id, exclusion_reason , participant_id
# NOTE: will be using rows only where exam == 1      ^ participant ids in rows w/ also exam==1; they are like 1###, matches with the "sidno" ids just below

In [ ]:
tmp <- read.csv("MESA_TOPMed_Metab_Proteo_BySidnoExam.csv")
dim(tmp)
colnames(tmp)
head(tmp)
# want: sidno, Exam, TOM_ID, TOP_ID, collaboratorsampleid, ageatdraw, bmi1c, race1c
# NOTE: also only exam 1 rows
table(tmp$race1c)

# (Ignore) MESA-WHI Combined File
Too much potential for human error, decided to do MESA and WHI separatelyto simplify things.

## Merge!

In [ ]:
# What maps what IDs:
  # bdl[[1]]: NWD <-> WHI <-> DBG
  # bdl[2:3]: NWD only
  # bld[[4]]: WHI <-> TOR <-> TOE <-> TOM
  # bld[[5]]: WHI <-> TOR
  # bld[[6]]: WHI <-> TOM
  # bld[[7]]: NWD <-> TOR <-> MESA
  # bld[[8]]: NWD <-> TOM <-> TOP <-> MESA

clin_df <- merge(x=bdl[[10]],all=T, by.x="subject_id",          # WHI (from WHI tech pheno w/ bmi)
        y=merge(x=bdl[[9]], all=T, by.x="subject_id",          # WHI (from meth+met visit timepoints)
       y=merge(x=bdl[[8]], all=T, by.x="collaboratorsampleid",# NWD (from MESA metabo-proteo)
      y=merge(x=bdl[[7]], all=T, by.x="nwd_id",              # NWD (from MESA RNASeq)
     y=merge(x=bdl[[6]], all=T, by.x="SUBJECT_ID",          # WHI (from WHI metabo)
    y=merge(x=bdl[[5]], all=T, by.x="SUBJECT_ID",          # WHI (from WHI RNASeq)
   y=merge(x=bdl[[4]], all=T, by.x="subject_id",          # WHI (from WHI tech pheno)
  y=merge(x=bdl[[3]], all=T, by.x="index",               # NWD (from kent's file)
 y=merge(x=bdl[[2]], all=T, by.x="X._VCF.Sample.Id.",   # NWD (from smoking)
        y=bdl[[1]],         by.y="sample.id"),          # NWD (from sgaynor)
                             by.y="X._VCF.Sample.Id."),  # NWD (from smoking)
                              by.y="subject_id"),         # WHI (from sgaynor)
                               by.y="subject_id"),         # WHI (from WHI tech pheno)
                                by.y="SUBJECT_ID"),         # WHI (from WHI RNASeq)
                                 by.y="index"),              # NWD (from kent's file)
                                  by.y="nwd_id"),             # NWD (from MESA RNASeq)
                                   by.y="SUBJECT_ID"),         # WHI (from WHI metabo)
                                    by.y="subject_id")          # WHI (from meth+met visit timepoints)
clin_df <- setDT(clin_df)
dim(clin_df); head(clin_df)

## Explore, determine which rows can be merged/removed

### CHECKPOINT: Check colnames and a random row

In [ ]:
rind <- trunc( runif(1,1,nrow(clin_df)) ); rind
print(clin_df[rind,])
colnames(clin_df); dim(clin_df)

### "collaboratorsampleid" vs. "topmedid" (NWD IDs)

In [ ]:
identical(clin_df[!is.na(topmedid), collaboratorsampleid], 
          clin_df[!is.na(topmedid), topmedid]) # Where topmedid is not NA, they are identical.
all(is.na(clin_df[is.na(collaboratorsampleid), topmedid])) # Where collaboratorsampleid is NA, so it topmedid. No new info.

clin_df <- clin_df[, topmedid := NULL] # They are identical except "topmedid" is less complete. Rm "topmedid".

### "subject_id" vs. "sidno" (NWD IDs)

In [ ]:
sum(is.na(clin_df$subject_id)); sum(is.na(clin_df$sidno))
clin_df[!is.na(sidno), sidno,subject_id][sidno!=subject_id] # Noooo! They (exluding NAs) are identical except for ~one~ conflict!
# Asked Magda; will trust the sgaynor file ("subject_id")

clin_df[is.na(subject_id) & !is.na(sidno),sidno] # However, sidno does have a couple ids where subject_id is NA, so grab those. But then rm sidno.
clin_df[is.na(subject_id), subject_id:=sidno]

clin_df <- clin_df[, sidno := NULL]

### "TOM_ID.x" (from MESA metabo) vs. "TOM_ID.y" (from WHI technical pheno)


In [ ]:
identical(clin_df[!is.na(TOM_ID.x) & !is.na(TOM_ID.y), TOM_ID.x],
          clin_df[!is.na(TOM_ID.x) & !is.na(TOM_ID.y), TOM_ID.y]) # Where they're both not NA, they agree. Safe to merge them.

clin_df[is.na(TOM_ID.x), TOM_ID.x := TOM_ID.y]
clin_df <- clin_df[, TOM_ID.y := NULL] # Rm "TOM_ID.y". Merged it into the .x one, so no longer needed.

### "tor_id" (from MESA RNASeq) vs. "SAMPLE_ID.y" (from WHI RNASeq)

In [ ]:
identical(clin_df[!is.na(tor_id) & !is.na(SAMPLE_ID.y), tor_id],
          clin_df[!is.na(tor_id) & !is.na(SAMPLE_ID.y), SAMPLE_ID.y]) # :] safe to merge

clin_df[is.na(tor_id), tor_id := SAMPLE_ID.y]
clin_df <- clin_df[, SAMPLE_ID.y := NULL] # Rm "SAMPLE_ID.y". Merged it into tor_id, so no longer needed.

### Remove rows which are NA for everything except smoking
There are a lot of these, and not so useful.

In [ ]:
tmp <- clin_df[, -c("collaboratorsampleid",
                    "X.DCC.Harmonized.data.set.03...Baseline.common.covariates.Indicates.whether.subject.ever.regularly.smoked.cigarettes..")]
rows2keep <- apply(tmp,1, function(row) !all(is.na(row)))

head( clin_df[!rows2keep], n=3) # How these NAs row look
print(paste(sum(!rows2keep),"unimportant rows will be removed, leaving", nrow(clin_df)-sum(!rows2keep)))
clin_df <- clin_df[rows2keep]
dim(clin_df)

### Remove rows which have no TOE,TOM,TOP,TOR ID

In [ ]:
dim(clin_df)
clin_df <- clin_df[ !( is.na(methylation_id) &
                       is.na(TOM_ID.x) & is.na(SAMPLE_ID.x) &
                       is.na(TOP_ID) &
                       is.na(tor_id) & is.na(rnaseq_id) )]
dim(clin_df)

### CHECKPOINT: Check colnames and a random row again

In [ ]:
# Check colnames and a random row
rind <- trunc( runif(1,1,nrow(clin_df)) ); rind
print(clin_df[rind,])
colnames(clin_df); dim(clin_df)

### Set TOE/TOM IDs to NA if they are from Baseline
Using Jeff's methylation & metabolomic visit timepoint info file. We just want the measurements not at baseline, to match the rest of the data we're using.\
Hopefully this will resolve some of the conflicts and duplicates in IDs?\
WARNING: note there are two different columns "methylation_id" and "methylaiton_id" \[sic\]\
\
I know there are still multiple columns for TOM and TOE IDs each, but there are conflicts and I'm not sure how to resolve them. Hoping that removing these baseline entires will remove some of the conflicts.

In [ ]:
table(clin_df$methylation_visit)
sum(is.na(clin_df$methylaiton_id)); sum(is.na(clin_df$methylation_id))
clin_df <- clin_df[methylation_visit=="Baseline", `:=`(methylation_id=NA, methylaiton_id=NA)]
sum(is.na(clin_df$methylaiton_id)); sum(is.na(clin_df$methylation_id))

table(clin_df$metabolomics_visit)
sum(is.na(clin_df$metabolomics_id)); sum(is.na(clin_df$TOM_ID.x)); sum(is.na(clin_df$SAMPLE_ID.x))
clin_df <- clin_df[metabolomics_visit=="Base", `:=`(metabolomics_id=NA, TOM_ID.x=NA, SAMPLE_ID.x=NA)]
sum(is.na(clin_df$metabolomics_id)); sum(is.na(clin_df$TOM_ID.x)); sum(is.na(clin_df$SAMPLE_ID.x))

clin_df <- clin_df[, `:=`(methylation_visit=NULL, metabolomics_visit=NULL)]

### "methylaiton_id" \[sic\] (from meth&met visit timepoint info) vs. "methylation_id" (from WHI technical phenotype)
Are the two methylation colunms identical now (exclusing NAs)? They were not before removing the baseline entires.

In [ ]:
identical(clin_df[!is.na(methylaiton_id) & !is.na(methylation_id), methylaiton_id],
          clin_df[!is.na(methylaiton_id) & !is.na(methylation_id), methylation_id]) # Yes!! :] safe to merge now.

clin_df[is.na(methylation_id), methylation_id := methylaiton_id]
clin_df <- clin_df[, methylaiton_id := NULL] # Rm "methylaiton_id" [sic]. Merged it into "methylation_id", so no longer needed.

### "metabolomics_id" (from meth&met visit timepoint info) vs. "TOM_ID.x" (from MESA metabo) vs. "SAMPLE_ID.x" (from WHI metabo)
Are the TOM colunms identical now (exclusing NAs)? They were not before removing the baseline entires (there were 105 conflicts between "TOM_ID.x" and "SAMPLE_ID.x").

In [ ]:
identical(clin_df[!is.na(metabolomics_id) & !is.na(TOM_ID.x), metabolomics_id],
          clin_df[!is.na(metabolomics_id) & !is.na(TOM_ID.x), TOM_ID.x]) # :] safe to merge
identical(clin_df[!is.na(SAMPLE_ID.x) & !is.na(TOM_ID.x), TOM_ID.x],
          clin_df[!is.na(SAMPLE_ID.x) & !is.na(TOM_ID.x), SAMPLE_ID.x]) # :] also good!

clin_df <- clin_df[is.na(TOM_ID.x), TOM_ID.x := metabolomics_id]
clin_df <- clin_df[is.na(TOM_ID.x), TOM_ID.x := SAMPLE_ID.x]

clin_df <- clin_df[, metabolomics_id := NULL]
clin_df <- clin_df[, SAMPLE_ID.x := NULL]

### "tor_id" (from MESA & WHI RNASeq) vs. "rnaseq_id" (from WHI technical pheno)

In [ ]:
identical(clin_df[!is.na(tor_id) & !is.na(rnaseq_id), tor_id],
          clin_df[!is.na(tor_id) & !is.na(rnaseq_id), rnaseq_id]) # conflicts!
head( clin_df[tor_id!=rnaseq_id, c("subject_id","tor_id","rnaseq_id")] )

# Jeff said for duplicate RNASeq samples, just pick one.
# Each conflict shown below is due to the same WHI ID  (in "subject_id") being associated w/ multiple TOR IDs (duplicated).
clin_df[tor_id!=rnaseq_id, rnaseq_id := tor_id] # Just pick the ones in tor_id.
clin_df[is.na(tor_id), tor_id := rnaseq_id] # Now can merge. And no longer need rnaseq_id
clin_df[, rnaseq_id := NULL]

# May need later?
#       dup_subject_ids <- unique(clin_df[         duplicated(subject_id),                   subject_id])
#dup_rnaseq_subject_ids <- unique(clin_df[tor_id!=rnaseq_id & subject_id%in%dup_subject_ids, subject_id])
#for(id in dup_rnaseq_subject_ids[1]) {
#    dup_row_inds <- which(clin_df$subject_id==id)
#    each_dup_n_na <- apply(clin_df[dup_row_inds],1, function(row) sum(is.na(row)))
#    rows2toss <- dup_row_inds[-which.min(each_dup_n_na)]
#    clin_df <- clin_df[-rows2toss]
#}

In [ ]:
identical(clin_df[!is.na(ageatdraw) & !is.na(draw_age), ageatdraw],
          clin_df[!is.na(ageatdraw) & !is.na(draw_age), draw_age]) # :] can merge
clin_df[is.na(ageatdraw), ageatdraw := draw_age]
clin_df[, draw_age := NULL]

## Prettier colnames

In [ ]:
nrow(clin_df)
colnames(clin_df)

In [ ]:
colnames(clin_df) <- c("Subject_Id",
                       "BMI",
                       "BMI_Age",
                       "NWD_Id",
                       "TOM_Id",
                       "TOP_Id",
                       "Age_at_Draw",
                       "BMI1c",
                       "TOR_Id",
                       "Exclusion_Reason",
                       "Subject_Fasting_Hours_WHI_Metabolomics",
                       "Subject_Fasting_Hours_WHI_RNASeq",
                       "Olink_Id",
                       "TOE_Id",
                       "Methylation_Batch",
                       "Methylation_Chip",
                       "Methylation_Row",
                       "RNASeq_Batch",
                       "Cohort",
                       "European",
                       "African",
                       "American",
                       "East_Asian",
                       "Smoking_Status",
                       "Sex",
                       "Ancestry",
                       "T2D",
                       "T2D_Age",
                       "T2D_BMI")
colnames(clin_df)

In [ ]:
dim(clin_df)
dim(clin_df[Cohort=="MESA"])
dim(clin_df[Cohort=="WHI"])
head(clin_df)
# TODO For covariates which are only in e.g. WHI, add "_WHI" to the colname or something
sum(duplicated(clin_df$Subject_Id))